# Pinecone
Pinecone은 고성능 벡터 데이터베이스로, AI 및 머신러닝 애플리케이션을 위한 효율적인 벡터 저장 및 검색 솔루션입니다.

Pinecone, Chroma, Faiss와 같은 벡터 데이터베이스들을 비교해보겠습니다.

**Pinecone의 장점**

1. 확장성: **대규모 데이터셋**에 대해 뛰어난 확장성을 제공합니다.

2. 관리 용이성: 완전 관리형 서비스로, 인프라 관리 부담이 적습니다.

3. 실시간 업데이트: 데이터의 실시간 삽입, 업데이트, 삭제가 가능합니다.

4. 고가용성: 클라우드 기반으로 높은 가용성과 내구성을 제공합니다.

5. API 친화적: RESTful/Python API를 통해 쉽게 통합할 수 있습니다.

**Pinecone의 단점**

1. 비용: Chroma나 Faiss에 비해 상대적으로 비용이 높을 수 있습니다.

2. 커스터마이징 제한: 완전 관리형 서비스이기 때문에 세부적인 커스터마이징에 제한이 있을 수 있습니다.

3. 데이터 위치: 클라우드에 데이터를 저장해야 하므로, 데이터 주권 문제가 있을 수 있습니다.

Chroma나 Faiss와 비교했을 때:

- Chroma/FAISS 오픈소스이며 로컬에서 실행 가능하여 초기 비용이 낮고 데이터 제어가 용이합니다. 커스터마이징의 자유도가 높습니다. 하지만 대규모 확장성 면에서는 Pinecone에 비해 제한적일 수 있습니다.
- 선택은 프로젝트의 규모, 요구사항, 예산 등을 고려하여 결정해야 합니다. 대규모 프로덕션 환경에서는 Pinecone이 유리할 수 있지만, 소규모 프로젝트나 실험적인 환경에서는 Chroma나 Faiss가 더 적합할 수 있습니다.

**참고**

- [Pinecone 공식 홈페이지](https://docs.pinecone.io/integrations/langchain)
- [Pinecone 랭체인](https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/)

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# 한국어 불용어 사전 
한글 불용어 사전 불러오기 (불용어 사전 출처: https://www.ranks.nl/stopwords/korean)

불용어는 큰 의미를 갖지 않는것을 의미한다. 이 불용어를 제거하고 retriever하게끔 만들어 준다.

In [2]:
from langchain_teddynote.korean import stopwords

stopword = stopwords()
stopword

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

In [3]:
len(stopword)

675

In [4]:
# 불용어 추가
stopword.append("를")

In [5]:
len(stopword)

676

# 데이터 전처리

In [6]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import glob

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

split_docs = []

# 텍스트 파일을 load -> List[Document] 형태로 변환
files = sorted(glob.glob("data/*.pdf"))
for file in files:
    loader = PDFPlumberLoader(file)
    split_docs.extend(loader.load_and_split(text_splitter))

len(split_docs)

619

In [7]:
split_docs[0].page_content

'ARAGOG: Advanced RAG Output Grading\nMatouˇs Eibich Shivay Nagpal Alexander Fred-Ojala\nPredli Predli Predli & UC Berkeley\nmatous@predli.com shivay@predli.com afo@berkeley.edu\nApril 2, 2024\nAbstract\nRetrieval-Augmented Generation (RAG) is essential for integrating external knowledge into\nLarge Language Model (LLM) outputs. While the literature on RAG is growing, it primarily\nfocusesonsystematicreviewsandcomparisonsofnewstate-of-the-art(SoTA)techniquesagainst\ntheirpredecessors,withagapinextensiveexperimentalcomparisons. Thisstudybeginstoaddress\nthisgapbyassessingvariousRAGmethods’impactsonretrievalprecisionandanswersimilarity.\nWefoundthatHypotheticalDocumentEmbedding(HyDE)andLLMrerankingsignificantlyen-\nhance retrieval precision. However, Maximal Marginal Relevance (MMR) and Cohere rerank did\nnot exhibit notable advantages over a baseline Naive RAG system, and Multi-query approaches\nunderperformed. SentenceWindowRetrievalemergedasthemosteffectiveforretrievalprecision,'

In [8]:
split_docs[0].metadata

{'source': 'data\\ARAGOG_Advanced RAG Output Grading.pdf',
 'file_path': 'data\\ARAGOG_Advanced RAG Output Grading.pdf',
 'page': 0,
 'total_pages': 14,
 'Author': '',
 'CreationDate': 'D:20240402041602Z',
 'Creator': 'LaTeX with hyperref',
 'Keywords': '',
 'ModDate': 'D:20240402041602Z',
 'PTEX.Fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
 'Producer': 'pdfTeX-1.40.25',
 'Subject': '',
 'Title': '',
 'Trapped': 'False'}

# 문서의 전처리
- 필요한 `metadata` 정보를 추출합니다.

- 최소 길이 이상의 데이만 필터링 합니다.

- 문서의 `basename` 을 사용할지 여부를 지정합니다. 기본값은 False 입니다.

    - 여기서 `basename` 이란 파일 경로의 가장 마지막 부분을 의미합니다.
    - 예를 들어, `/Users/teddy/data/document.pdf` 의 경우 `document.pdf` 가 됩니다.

In [9]:
split_docs[0].page_content

'ARAGOG: Advanced RAG Output Grading\nMatouˇs Eibich Shivay Nagpal Alexander Fred-Ojala\nPredli Predli Predli & UC Berkeley\nmatous@predli.com shivay@predli.com afo@berkeley.edu\nApril 2, 2024\nAbstract\nRetrieval-Augmented Generation (RAG) is essential for integrating external knowledge into\nLarge Language Model (LLM) outputs. While the literature on RAG is growing, it primarily\nfocusesonsystematicreviewsandcomparisonsofnewstate-of-the-art(SoTA)techniquesagainst\ntheirpredecessors,withagapinextensiveexperimentalcomparisons. Thisstudybeginstoaddress\nthisgapbyassessingvariousRAGmethods’impactsonretrievalprecisionandanswersimilarity.\nWefoundthatHypotheticalDocumentEmbedding(HyDE)andLLMrerankingsignificantlyen-\nhance retrieval precision. However, Maximal Marginal Relevance (MMR) and Cohere rerank did\nnot exhibit notable advantages over a baseline Naive RAG system, and Multi-query approaches\nunderperformed. SentenceWindowRetrievalemergedasthemosteffectiveforretrievalprecision,'

In [10]:
split_docs[0].metadata

{'source': 'data\\ARAGOG_Advanced RAG Output Grading.pdf',
 'file_path': 'data\\ARAGOG_Advanced RAG Output Grading.pdf',
 'page': 0,
 'total_pages': 14,
 'Author': '',
 'CreationDate': 'D:20240402041602Z',
 'Creator': 'LaTeX with hyperref',
 'Keywords': '',
 'ModDate': 'D:20240402041602Z',
 'PTEX.Fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
 'Producer': 'pdfTeX-1.40.25',
 'Subject': '',
 'Title': '',
 'Trapped': 'False'}

In [33]:
# 저장하고 싶은 메타데이터만 뽑는다.
from langchain_teddynote.community.pinecone import preprocess_documents

contents, metadatas = preprocess_documents(
    split_docs=split_docs,
    #metadata_keys=["source", "page", "author"],
    metadata_keys=["source", "page"],
    min_length=20, #문서의 길이를 가지고 filtering. 5글자 이상의 문서만을 사용. (예를들어 숫자만 있거나 제목만 있는경우) 줄어든것을 확인할 수 있다.
    use_basename=True # False인 경우 폴더 위치까지 모두 적혀있다.
)

  0%|          | 0/619 [00:00<?, ?it/s]

In [34]:
len(contents)

617

In [35]:
contents[:5]

['ARAGOG: Advanced RAG Output Grading\nMatouˇs Eibich Shivay Nagpal Alexander Fred-Ojala\nPredli Predli Predli & UC Berkeley\nmatous@predli.com shivay@predli.com afo@berkeley.edu\nApril 2, 2024\nAbstract\nRetrieval-Augmented Generation (RAG) is essential for integrating external knowledge into\nLarge Language Model (LLM) outputs. While the literature on RAG is growing, it primarily\nfocusesonsystematicreviewsandcomparisonsofnewstate-of-the-art(SoTA)techniquesagainst\ntheirpredecessors,withagapinextensiveexperimentalcomparisons. Thisstudybeginstoaddress\nthisgapbyassessingvariousRAGmethods’impactsonretrievalprecisionandanswersimilarity.\nWefoundthatHypotheticalDocumentEmbedding(HyDE)andLLMrerankingsignificantlyen-\nhance retrieval precision. However, Maximal Marginal Relevance (MMR) and Cohere rerank did\nnot exhibit notable advantages over a baseline Naive RAG system, and Multi-query approaches\nunderperformed. SentenceWindowRetrievalemergedasthemosteffectiveforretrievalprecision,',
 '

In [36]:
metadatas.keys()



dict_keys(['source', 'page'])

In [37]:
metadatas["source"][:5]

['ARAGOG_Advanced RAG Output Grading.pdf',
 'ARAGOG_Advanced RAG Output Grading.pdf',
 'ARAGOG_Advanced RAG Output Grading.pdf',
 'ARAGOG_Advanced RAG Output Grading.pdf',
 'ARAGOG_Advanced RAG Output Grading.pdf']

In [38]:
# 내용 3개가 맞아 떨어지는지 확인
len(contents), len(metadatas["source"]), len(metadatas['page'])

(617, 617, 617)

# 새로운 VectorStore 인덱스 생성
Pinecone 의 새로운 인덱스를 생성합니다.

pinecone 인덱스를 생성합니다.

**주의사항**

- metric 은 유사도 측정 방법을 지정합니다. 만약 HybridSearch 를 고려하고 있다면 metric 은 dotproduct 로 지정합니다.

In [39]:
import os
from langchain_teddynote.community.pinecone import create_index

pc_index = create_index(
    api_key=os.environ["PINECONE_API_KEY"],
    index_name="jaeho-note-index", # 인덱스 이름을 지정합니다.
    dimension=1536, # Embedding 차원과 맞춥니다. (OpenAIEmbeddings: 1536, UpstageEmbeddings: 4096)
    metric="dotproduct" # 유사도 측정 방법을 지정합니다. (dotproduct, euclidean, cosine) Hybrid search를 위해선 dotproduct를 사용
)

[create_index]
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'jaehonote-namespace': {'vector_count': 0}},
 'total_vector_count': 0}


In [40]:
# 아래는 유료 Pod 를 사용하는 예시입니다. 유료 Pod 는 무료 Serverless Pod 대비 더 확장된 기능을 제공합니다.

# 참고: https://docs.pinecone.io/guides/indexes/choose-a-pod-type-and-size
# import os
# from langchain_teddynote.community.pinecone import create_index
# from pinecone import PodSpec

# # Pinecone 인덱스 생성
# pc_index = create_index(
#     api_key=os.environ["PINECONE_API_KEY"],
#     index_name="jaehonote",  # 인덱스 이름을 지정합니다.
#     dimension=1536,  # Embedding 차원과 맞춥니다. (OpenAIEmbeddings: 1536, UpstageEmbeddings: 4096)
#     metric="dotproduct",  # 유사도 측정 방법을 지정합니다. (dotproduct, euclidean, cosine)
#     pod_spec=PodSpec(
#         environment="us-west1-gcp", pod_type="p1.x1", pods=1
#     ),  # 유료 Pod 사용
# )

In [41]:
from langchain_openai import OpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings



# Sparse Encoder 생성
- Sparse Encoder 를 생성합니다.
- `Kiwi Tokenizer` 와 한글 불용어(stopwords) 처리를 수행합니다. 내재화 되어있는 인코더는 영어로 맞춰줘 있다. 한글로 맞추기 위해 사용한다.
- Sparse Encoder 를 활용하여 contents 를 학습합니다. 여기서 학습한 인코드는 VectorStore 에 문서를 저장할 때 Sparse Vector 를 생성할 때 활용합니다.

In [42]:
from langchain_teddynote.korean import stopwords
from langchain_teddynote.community.pinecone import create_sparse_encoder, fit_sparse_encoder

stopword = stopwords()
stopword

# 한글 불용어 사전 stopwords() + Kiwi 형태소 분석기
sparse_encoder = create_sparse_encoder(stopwords(), mode='kiwi') # 한글처리를 하기위한 kiwi

Sparse Encoder를 Corpus에 학습한다.
- `save_path`: Sparse Encoder를 저장할 경로. 추후에 `pikcle`형식으로 저장한 Sparse Encoder를 불러와 Query임베딩할떄 사용한다. 따라서 저장할 경로를 지정한다.

In [43]:
# Sparse Encoder를 사용하여 contents를 학습
save_path = fit_sparse_encoder(
    sparse_encoder=sparse_encoder, contents=contents, save_path="./sparse_encoder.pkl"
)

  0%|          | 0/617 [00:00<?, ?it/s]

[fit_sparse_encoder]
Saved Sparse Encoder to: ./sparse_encoder.pkl


[선택사항] 나중에 학습을 저장한 Sparse Encoder를 다시 불러올떄 사용하는 코드

In [44]:
# 저장한 sparse encoder불러오기
from langchain_teddynote.community.pinecone import load_sparse_encoder

sparse_encoder = load_sparse_encoder("./sparse_encoder.pkl")

[load_sparse_encoder]
Loaded Sparse Encoder from: ./sparse_encoder.pkl


# Pinecone: DB Index에 추가 (Upsert)
Upsert: update + insert
- `context`: 문서의 내용입니다.
- `page`: 문서의 페이지 번호입니다.
- `source`: 문서의 출처입니다.
- `values`: Embedder 를 통해 얻은 문서의 임베딩입니다.
- `sparse values`: Sparse Encoder 를 통해 얻은 문서의 임베딩입니다.

In [47]:
#%%time
from langchain_teddynote.community.pinecone import upsert_documents
from langchain_openai import OpenAIEmbeddings
openai_embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
upsert_documents(
    index=pc_index, #picone 인덱스
    namespace="jaehonote-namespace-0", #파인콘 namespace. 비슷한 문서에는 같은 namespace에 저장
    contents=contents, # 이전에 처리한 문서내용
    metadatas=metadatas, # 이전에 처리한 문서 데이터 
    sparse_encoder=sparse_encoder, # Sparse Encoder
    embedder=openai_embeddings,
    batch_size=32,
    
)

  0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

분산처리

In [54]:
#%%time
from langchain_teddynote.community.pinecone import upsert_documents
from langchain_openai import OpenAIEmbeddings
from langchain_teddynote.community.pinecone import upsert_documents_parallel

openai_embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
upsert_documents_parallel(
    index=pc_index, #picone 인덱스
    namespace="jaehonote-namespace-1", #파인콘 namespace. 비슷한 문서에는 같은 namespace에 저장
    contents=contents, # 이전에 처리한 문서내용
    metadatas=metadatas, # 이전에 처리한 문서 데이터 
    sparse_encoder=sparse_encoder, # Sparse Encoder
    embedder=openai_embeddings,
    batch_size=64, #chunk size * batch_size. 값이 너무크면 에러가난다. 1000*64
    max_workers=30, # 위 업데이트를 30명이 동시에 처리하는것. 1000*64을 30명에서 한다고 생각하면됌. 엄청빠름! 약 1분을 6초로 줄여버렸다!
)

문서 Upsert 중:   0%|          | 0/10 [00:00<?, ?it/s]

총 617개의 Vector 가 Upsert 되었습니다.
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'jaehonote-namespace': {'vector_count': 882}},
 'total_vector_count': 882}


# 인덱스 조회/삭제
`describe_index_stats` 메서드는 인덱스의 내용에 대한 통계 정보를 제공합니다. 이 메서드를 통해 네임스페이스별 벡터 수와 차원 수 등의 정보를 얻을 수 있습니다.

**매개변수**
- filter (Optional[Dict[str, Union[str, float, int, bool, List, dict]]]): 특정 조건에 맞는 벡터들에 대한 통계만 반환하도록 하는 필터. 기본값은 None
- **kwargs: 추가 키워드 인자

**반환값**
- DescribeIndexStatsResponse: 인덱스에 대한 통계 정보를 담고 있는 객체

**사용 예시**
- 기본 사용: index.describe_index_stats()
- 필터 적용: index.describe_index_stats(filter={'key': 'value'})

**참고**
- metadata 필터링은 유료 사용자에 한하여 가능합니다.

In [55]:
# 인덱스 조회
pc_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'jaehonote-namespace': {'vector_count': 1458}},
 'total_vector_count': 1458}

## 네임스페이스 삭제

In [ ]:
from langchain_teddynote.community.pinecone import delete_namespace

delete_namespace(
    pinecone_index=pc_index,
    namespace="jaehonote-namespace-0"
)

In [ ]:
# 인덱스 조회
pc_index.describe_index_stats()

유료사용자 전용 기능: metadata의 filter기능. 

$eq은 일치기능 SPRi AI Brief_8월호_산업동향.pdf만 지워줌

In [ ]:
from langchain_teddynote.community.pinecone import delete_by_filter

# metadata 필터링(유료 기능) 으로 삭제
delete_by_filter(
    pinecone_index=pc_index,
    namespace="teddynote-namespace-01",
    filter={"source": {"$eq": "SPRi AI Brief_8월호_산업동향.pdf"}},
)
pc_index.describe_index_stats()

## 검색기(Retriever) 생성

### PineconeKiwiHybridRetriever 초기화 파라미터 설정

`init_pinecone_index` 함수와 `PineconeKiwiHybridRetriever` 클래스는 Pinecone을 사용한 하이브리드 검색 시스템을 구현합니다. 이 시스템은 밀집 벡터와 희소 벡터를 결합하여 효과적인 문서 검색을 수행합니다.

**Pinecone 인덱스 초기화**

`init_pinecone_index` 함수는 Pinecone 인덱스를 초기화하고 필요한 구성 요소를 설정합니다.

**매개변수**
* `index_name` (str): Pinecone 인덱스 이름
* `namespace` (str): 사용할 네임스페이스
* `api_key` (str): Pinecone API 키
* `sparse_encoder_pkl_path` (str): 희소 인코더 피클 파일 경로
* `stopwords` (List[str]): 불용어 리스트
* `tokenizer` (str): 사용할 토크나이저 (기본값: "kiwi")
* `embeddings` (Embeddings): 임베딩 모델
* `top_k` (int): 반환할 최대 문서 수 (기본값: 10)
* `alpha` (float): 밀집 벡터와 희소 벡터의 가중치 조절 파라미터 (기본값: 0.5)

**주요 기능**
1. Pinecone 인덱스 초기화 및 통계 정보 출력
2. 희소 인코더(BM25) 로딩 및 토크나이저 설정
3. 네임스페이스 지정

In [61]:
from langchain_teddynote.community.pinecone import init_pinecone_index

pinecone_params = init_pinecone_index(
    index_name="jaeho-note-index",  # Pinecone 인덱스 이름
    namespace="jaehonote-namespace",  # Pinecone Namespace
    api_key=os.environ["PINECONE_API_KEY"],  # Pinecone API Key
    sparse_encoder_path="./sparse_encoder.pkl",  # Sparse Encoder 저장경로(save_path)
    stopwords=stopwords(),  # 불용어 사전
    tokenizer="kiwi",
    embeddings=openai_embeddings,  # Dense Embedder
    top_k=5,  # Top-K 문서 반환 개수
    alpha=0.5,  # alpha=0.75로 설정한 경우, (0.75: Dense Embedding, 0.25: Sparse Embedding)
)

[init_pinecone_index]
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'jaehonote-namespace': {'vector_count': 1458}},
 'total_vector_count': 1458}


### PineconeKiwiHybridRetriever

`PineconeKiwiHybridRetriever` 클래스는 Pinecone과 Kiwi를 결합한 하이브리드 검색기를 구현합니다.

**주요 속성**
* `embeddings`: 밀집 벡터 변환용 임베딩 모델
* `sparse_encoder`: 희소 벡터 변환용 인코더
* `index`: Pinecone 인덱스 객체
* `top_k`: 반환할 최대 문서 수
* `alpha`: 밀집 벡터와 희소 벡터의 가중치 조절 파라미터
* `namespace`: Pinecone 인덱스 내 네임스페이스

**특징**
* 밀집 벡터와 희소 벡터를 결합한 HybridSearch Retriever
* 가중치 조절을 통한 검색 전략 최적화 가능
* 다양한 동적 metadata 필터링 적용 가능(`search_kwargs` 사용: `filter`, `k`, `rerank`, `rerank_model`, `top_n` 등)

**사용 예시**
1. `init_pinecone_index` 함수로 필요한 구성 요소 초기화
2. 초기화된 구성 요소로 `PineconeKiwiHybridRetriever` 인스턴스 생성
3. 생성된 검색기를 사용하여 하이브리드 검색 수행

In [62]:
from langchain_teddynote.community.pinecone import PineconeKiwiHybridRetriever

# 검색기 생성
pinecone_retriever = PineconeKiwiHybridRetriever(**pinecone_params)

In [63]:
# 실행 결과
search_results = pinecone_retriever.invoke("gpt-4o 미니 출시 관련 정보에 대해서 알려줘")
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육
갈릴레오의 LLM 환각 지수 평가에서 GPT-4가 가장 우수
KEY Contents
n 주요 LLM의 환각 현상을 평가한 ‘LLM 환각 지수’에 따르면 GPT-4는 작업 유형과 관계없이
가장 우수한 성능을 보였으며 GPT-3.5도 거의 동등한 성능을 발휘
n 오픈소스 모델 중에서는 메타의 라마2가 RAG 없는 질문과 답변 및 긴 형식의 텍스트
생성에서 가장 우수한 성능을 발휘
£주요 LLM 중 GPT-4가 가장 환각 현상 적고 GPT-3.5 터보도 비슷한 성능 기록
n 머신러닝 데이터 관리 기업 갈릴레오(Galileo)가 2023년 11월 15일 주요 LLM의 환각 현상을 평가한
‘LLM 환각 지수(LLM Hallucination Index)’를 발표
∙ 생성 AI의 환각 현상은 AI 시스템이 잘못된 정보를 생성하거나, 현실과 다른 부정확한 결과를 내놓는
현상으로, 기업의 AI 도입을 가로막는 주요 장애물이며, 환각 지수는 신뢰할 수 있는 생성 AI 구축을 위해
환각을 평가하고 측정하는 구조화된 접근방식을 제공
∙ 환각 지수는 △검색 증강 생성(Retrieval-Augmented Generation, RAG)*을 포함한 질문과 답변 △RAG
없는 질문과 답변 △긴 형식의 텍스트(보고서나 기사, 에세이) 생성의 3개 작업 유형에 대하여 환각을
기준으로 LLM의 순위를 평가
* 기존에 학습된 데이터가 아닌 외부 소스(데이터셋, 데이터베이스, 문서 등)에서 가져온 정보를 검색해 활용하는 기술
n 3개의 작업 유형 평가 전체에서 오픈AI의 GPT-4가 최고의 성능을 기록했으며, GPT-3.5 터보도
GPT-4와 거의 동등한 성능을 발휘
∙ 메타의 라마2(Llama-2-70b)는 RAG 없는 질문과 답변 유형에서 오픈소스 모델 가운데 가장 우수했고 긴
형식의 텍스트 생성에서도 GPT-4에 준하는 성능을 기록했으나, RAG 포함 질문과 답변에서는 허깅
페이스의 제퍼(Zephyr-7b)가 라마2를 능가
{'p

In [64]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽", search_kwargs={"alpha": 1, "k": 1}
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육
구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화
KEY Contents
n 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은
구글과 클라우드 서비스 사용 계약도 체결
n 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인
앤스로픽 및 오픈AI와 협력을 확대하는 추세
£구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공
n 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억
달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침
∙ 구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월
앤스로픽에 최대 40억 달러의 투자 계획을 공개
∙ 한편, 2023년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해
4년간 30억 달러 규모의 계약을 체결
∙ 오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei)
남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발
n 아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인 오픈
AI와 협력을 확대
∙ 마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023년 1월 추가로 100억 달러를
투자하기로 하면서 오픈AI의 지분 49%를 확보했으며, 오픈AI는 마이크로소프트의 애저(Azure)
클라우드 플랫폼을 사용해 AI 모델을 훈련
£구글, 클라우드 경쟁력 강화를 위해 생성 AI 투자 확대
n 구글은 수익률이 높은 클라우드 컴퓨팅 시장에서 아마존과 마이크로소프트를 따라잡고자 생성 AI를
통한 기업 고객의 클라우드 지출 확대를 위해 AI 투자를 지속
{'page': 13.0, 'source'

In [65]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽", search_kwargs={"alpha": 0, "k": 1}
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육
구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화
KEY Contents
n 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은
구글과 클라우드 서비스 사용 계약도 체결
n 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인
앤스로픽 및 오픈AI와 협력을 확대하는 추세
£구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공
n 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억
달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침
∙ 구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월
앤스로픽에 최대 40억 달러의 투자 계획을 공개
∙ 한편, 2023년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해
4년간 30억 달러 규모의 계약을 체결
∙ 오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei)
남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발
n 아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인 오픈
AI와 협력을 확대
∙ 마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023년 1월 추가로 100억 달러를
투자하기로 하면서 오픈AI의 지분 49%를 확보했으며, 오픈AI는 마이크로소프트의 애저(Azure)
클라우드 플랫폼을 사용해 AI 모델을 훈련
£구글, 클라우드 경쟁력 강화를 위해 생성 AI 투자 확대
n 구글은 수익률이 높은 클라우드 컴퓨팅 시장에서 아마존과 마이크로소프트를 따라잡고자 생성 AI를
통한 기업 고객의 클라우드 지출 확대를 위해 AI 투자를 지속
{'page': 13.0, 'source'

동적 `search_kwargs` 사용
- `filter`: metadata 필터링 적용

(예시) `page` 가 5보다 작은 문서만 검색합니다.

In [66]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽의 claude 출시 관련 내용을 알려줘",
    search_kwargs={"filter": {"page": {"$lt": 5}}, "k": 2},
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

2023년 12월호
Ⅰ
. 인공지능 산업 동향 브리프
1. 정책/법제
▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 ·························1
▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의···························2
▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언···························3
▹ 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각·····································4
▹ 미국 연방거래위원회, 저작권청에 소비자 보호와 경쟁 측면의 AI 의견서 제출·················5
▹ EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항···················································6
2. 기업/산업
▹ 미국 프런티어 모델 포럼, 1,000만 달러 규모의 AI 안전 기금 조성································7
▹ 코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개 ·······································8
▹ 알리바바 클라우드, 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································9
▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ···························································10
▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································11
▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망···········································12
{

동적 `search_kwargs` 사용
- `filter`: metadata 필터링 적용

(예시) `source` 가 `SPRi AI Brief_8월호_산업동향.pdf` 문서내 검색합니다.

In [67]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽의 claude 3.5 출시 관련 내용을 알려줘",
    search_kwargs={
        "filter": {"source": {"$eq": "SPRi AI Brief_7월호_산업동향.pdf"}},
        "k": 3,
    },
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

### Reranking 적용

아직은 `pre` 기능입니다.

- 동적 reranking 기능을 구현해 놓았지만, pinecone 라이브러리 의존성에 문제가 있을 수 있습니다.
- 따라서, 아래 코드는 향후 의존성 해결 후 원활하게 동작할 수 있습니다.

참고 문서: https://docs.pinecone.io/guides/inference/rerank

In [68]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽의 클로드 소넷",
    search_kwargs={"rerank": True, "rerank_model": "bge-reranker-v2-m3", "top_n": 3},
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

[rerank_documents]


AttributeError: 'GRPCIndex' object has no attribute 'inference'